# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import ssl
import nltk
import re
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
columns = ['target','id','date','flag','user','text']

df = pd.read_csv('sentiment_analysis.csv', names=columns, encoding='latin-1')

In [3]:
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [37]:
sample = df.sample(n=20000, replace=False)

In [10]:
sample

,target,id,date,flag,user,text
625399,0,2230229086,Thu Jun 18 16:49:13 PDT 2009,NO_QUERY,MichealAlex,Going to Work
762965,0,2298106099,Tue Jun 23 11:15:48 PDT 2009,NO_QUERY,k3online,@trackle I am ready #trackle but always ending...
1341184,4,2031327354,Thu Jun 04 09:39:41 PDT 2009,NO_QUERY,Cat_oO,@FreakiLiCiOuS byee
1509087,4,2174752613,Sun Jun 14 23:14:07 PDT 2009,NO_QUERY,Dannifresh,@ChaseNCashe Some of us girls jus believe n th...
1463769,4,2064167871,Sun Jun 07 05:26:52 PDT 2009,NO_QUERY,pawnman44004,@ChessWhiz Great thing to remember when all yo...
...,...,...,...,...,...,...
500960,0,2187014674,Mon Jun 15 19:39:57 PDT 2009,NO_QUERY,rolltide007,@JKsNaughtygirl Good luck! I'll be at the Vega...
448877,0,2069103641,Sun Jun 07 15:16:05 PDT 2009,NO_QUERY,Cherrim,Why didn't I follow a guide of sorts for OoT? ...
638963,0,2234599749,Thu Jun 18 23:22:05 PDT 2009,NO_QUERY,shizie,@legxxxlove_slug apparently my pc and your mac...
905048,4,1695001108,Mon May 04 04:53:07 PDT 2009,NO_QUERY,jasonaltenburg,I'm going to be doing the FAFSA form today. I...


In [6]:
# I tried importing the functions from my other notebook but they kept giving me the wrong results...

# %run challenge_1.ipynb import clean_up, tokenize, stem_and_lemmatize, remove_stopwords

In [18]:
def clean_up(text):
    cleaned = ''.join([number for number in text if not number.isdigit()])
    cleaned = re.sub(r'http\S+', '', text)
    cleaned = re.sub('[^\w\s]', ' ', text).lower()
    return cleaned

def tokenize(cleaned):
    tokenized = word_tokenize(cleaned)
    return tokenized

def stem_and_lemmatize(tokenized):
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in tokenized]
    return lemmatized

def remove_stopwords(lemmatized):
    remove_sw = [word for word in lemmatized if not word in stopwords.words()]
    return remove_sw

In [38]:
text_processed = []

for text in sample['text']:
    cleaned = clean_up(text)
    tokenized = tokenize(cleaned)
    lemmatized = stem_and_lemmatize(tokenized)
    remove_sw = remove_stopwords(lemmatized)    
    text_processed.append(remove_sw)

sample['text_processed'] = text_processed

In [39]:
sample

,target,id,date,flag,user,text,text_processed
48554,0,1677904005,Sat May 02 04:35:46 PDT 2009,NO_QUERY,tsm121,Sending my MacBook Pro to service. Going to be...,"[sending, macbook, pro, service, tweet]"
781595,0,2323454211,Thu Jun 25 00:40:52 PDT 2009,NO_QUERY,bankslikeashley,@datdudepooch O so ya'll are really gonna be a...,"[datdudepooch, gon, area, homeboy, promoting, ..."
1102638,4,1970952898,Sat May 30 05:46:12 PDT 2009,NO_QUERY,PreternReviews,@anyabast Told you it was funny,"[anyabast, told, wa, funny]"
1147599,4,1978093993,Sat May 30 22:25:36 PDT 2009,NO_QUERY,dinstar,I just figured out that twitter sucks at mathe...,"[figured, twitter, suck, mathematics]"
371349,0,2050403646,Fri Jun 05 18:36:26 PDT 2009,NO_QUERY,trenddoll,@MissMama - WITHOUT ME NO FAIR!,"[missmama, fair]"
...,...,...,...,...,...,...,...
509411,0,2189399514,Mon Jun 15 23:56:29 PDT 2009,NO_QUERY,ElleVxo,"Trying to import e-mail contacts for twitter, ...","[import, mail, contact, twitter, working]"
726683,0,2262637056,Sat Jun 20 22:54:59 PDT 2009,NO_QUERY,andreaaa_tiger,I miss my baby. Distance is harder than anyo...,"[miss, baby, distance, harder, possibly, imagi..."
1559218,4,2186014012,Mon Jun 15 18:13:58 PDT 2009,NO_QUERY,TEAM_BSB_KTBSPA,@angelstorm00 lol thank you for suggesting us ...,"[angelstorm00, suggesting, bsb]"
546268,0,2201702116,Tue Jun 16 20:39:01 PDT 2009,NO_QUERY,MsFoxxymethod,i done missed my fly time with @OfficialROME s...,"[missed, fly, time, officialrome, sooo, pissed..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [49]:
from nltk.probability import ConditionalFreqDist

all_words = []

for lst in sample['text_processed']:
     for word in lst:
         all_words.append(word)

all_words = nltk.FreqDist(all_words)

bag_words = list(all_words.keys())[:5000]

In [61]:
all_words

FreqDist({'day': 1377, 'wa': 1279, 'quot': 939, 'today': 917, 'love': 880, 'http': 877, 'time': 842, 'work': 831, 'back': 758, 'amp': 586, ...})

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [66]:
def find_features(document):
    words = set(document)
    features = {}
    for w in bag_words:
        features[w] = (w in words)

    return features

In [78]:
feature_set = [(find_features(tweet), target) for (tweet, target) in list(zip(sample['text_processed'], sample['target']))]

In [ ]:
feature_set[0]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

In [80]:
training_data = feature_set[:2500]
test_data = feature_set[2500:]

classifier = nltk.NaiveBayesClassifier.train(training_data)

In [81]:
classifier.show_most_informative_features()

Most Informative Features
                    haha = True                4 : 0      =     13.8 : 1.0
                 amazing = True                4 : 0      =     10.1 : 1.0
                    poor = True                0 : 4      =      9.9 : 1.0
                    suck = True                0 : 4      =      7.6 : 1.0
                    miss = True                0 : 4      =      6.9 : 1.0
                tommcfly = True                4 : 0      =      6.8 : 1.0
                 anymore = True                0 : 4      =      6.5 : 1.0
                     boo = True                0 : 4      =      6.5 : 1.0
                   broke = True                0 : 4      =      6.5 : 1.0
                     cry = True                0 : 4      =      6.5 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [82]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_data))*100)

Classifier accuracy percent: 66.98857142857143


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here